<a href="https://colab.research.google.com/github/amlua/bi-master/blob/carga-analise-dados/download_preparacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Composição de Carteira de Fundos de Investimento: Obtenção e preparação dos dados
---


## Importação de bibliotecas

In [1]:
# import pandas e numpy
import pandas as pd
import numpy as np

In [2]:
# import google.colab.drive
from google.colab import drive

In [3]:
# import os, glob, zipfile e hashlib
import os
import glob
import zipfile
import hashlib

In [4]:
# import datetime, timedelta e time
from datetime import datetime
from datetime import timedelta
import time

### Selenium

In [ ]:
# Instalação do navegador e do pacote Selenium
!apt install chromium-chromedriver
!pip install selenium

In [6]:
# import webdriver e  ActionChains (selenium)
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

## Definição de funções

In [7]:
url_dados_abertos_cvm = 'http://dados.cvm.gov.br/dados'

def comptc():
  hoje = datetime.now() - timedelta(days=7)
  ano_atual = hoje.year
  mes_atual = hoje.month
  mes_comptc = mes_atual - 4 
  ano_comptc = ano_atual
  if mes_comptc <= 0:
    ano_comptc = ano_comptc -1
    mes_comptc = 12 + mes_comptc
  return '_{:02d}{:02d}'.format(ano_comptc,mes_comptc)

def busca_dados_abertos_cvm_cadastro():
  return url_dados_abertos_cvm + '/FI/CAD/DADOS/cad_fi.csv'

def busca_dados_abertos_cvm_carteira():
  arquivo = '/FI/DOC/CDA/DADOS/cda_fi'
  url = '{}{}'.format(url_dados_abertos_cvm,arquivo)
  url = url + comptc() + '.zip'
  return url

In [8]:
def busca_isin_b3():
  url_busca_isin_b3 = 'https://sistemaswebb3-listados.b3.com.br/isinPage/?language=pt-br'

  # Configuração do webdriver
  options = webdriver.ChromeOptions()
  options.add_argument('-headless')
  options.add_argument('-no-sandbox')
  options.add_argument('-disable-dev-shm-usage')
  wd = webdriver.Chrome('chromedriver',options=options)

  # Acesso à página
  wd.get(url_busca_isin_b3)

  # Interação com a página e download do arquivo isinp.zip
  button1 = wd.find_element_by_class_name('collapsed')
  button1.click()
  button3 = wd.find_element_by_class_name('click-p')
  action=ActionChains(wd)
  time.sleep(10)
  action.move_to_element(button3).click().perform();
  time.sleep(20)
 

In [9]:
def formataCNPJ(cnpj):
  if cnpj == '':
    return cnpj
  cnpj = str(cnpj)
  if len(cnpj) < 14:
    cnpj = cnpj.zfill(14)
  return '{}.{}.{}/{}-{}'.format(cnpj[:2], cnpj[2:5], cnpj[5:8], cnpj[8:12], cnpj[12:])

In [10]:
def aplic_ativo(row):
  return row['TP_APLIC'] + ' - ' + row['TP_ATIVO']

def aplic_ativo_descricao(row):
  return aplic_ativo(row) + ' - ' + row['DS_ATIVO']

In [11]:
def sit_aux(row):
  if row['SIT'] in ['CANCELADA', 'INCORPORAÇÃO', 'TRANSFORMAÇÃO']:
    return 0
  else:
    return 1

In [12]:
def ds_ativo(row):
  if row['DS_ATIVO_2'] != '':
    return row['DS_ATIVO_2']
  else:
    return row['DS_ATIVO_1']

In [13]:
def emissor(row):
  if row['EMISSOR_2'] != '':
    return row['EMISSOR_2']
  else:
    return row['EMISSOR_1']

In [14]:
def pr_patrim_liq(row):
  a = float(row['VL_APLIC'])
  b = float(row['VL_PATRIM_LIQ'])
  if a == 0 or b == 0:
    return 0
  else:
    if row['TP_APLIC'] in ['Opções - Posições lançadas', 'Valores a pagar', 'Obrigações por ações e outros TVM recebidos em empréstimo', 'DIFERENCIAL DE SWAP A PAGAR']:
      return round(a/b * (-100),3)
    else:
      return round(a/b * (100),3)  

In [15]:
def id_emissor(row):
  if row['CPF_CNPJ_EMISSOR']!='':
    return row['CPF_CNPJ_EMISSOR'] + '|'
  elif row['EMISSOR']!='':
    return '|' + row['EMISSOR']
  else:
    return ''

In [16]:
def hash(frase):
  if frase == '':
    return frase
  else:
    return hashlib.md5(str(frase).encode('utf-8')).hexdigest()   

## Download dos arquivos de dados


In [ ]:
# Cadastro de fundos (PDA-CVM)
url = busca_dados_abertos_cvm_cadastro()
os.system('wget %s -O ./cad_fi.csv' %url)

In [ ]:
# Carteiras de fundos (PDA-CVM)
url = busca_dados_abertos_cvm_carteira()
os.system('wget %s -O ./cda_fi.zip' %url)

In [19]:
# Emissores e ativos (B3)
busca_isin_b3()

In [ ]:
!ls -l

### Datasets de informações  cadastrais de fundos (Portal de Dados Abertos - CVM)

In [21]:
cad_fundo       = pd.read_csv('./cad_fi.csv', sep=';', encoding='ISO-8859-1',
                              usecols=['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT', 'DT_INI_SIT', 'TP_FUNDO', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO', 'CPF_CNPJ_GESTOR'], dtype=object)
cad_gestor      = pd.read_csv('./cad_fi.csv', sep=';', encoding='ISO-8859-1',
                              usecols=['CPF_CNPJ_GESTOR','GESTOR'], dtype=object)

In [22]:
cad_fundo['FUNDO_COTAS'].replace('N', 'False', inplace=True)
cad_fundo['FUNDO_COTAS'].replace('S', 'True', inplace=True)

cad_fundo['FUNDO_EXCLUSIVO'].replace('N', 'False', inplace=True)
cad_fundo['FUNDO_EXCLUSIVO'].replace('S', 'True', inplace=True)

### Datasets de aplicações de fundos (Portal de Dados Abertos - CVM)


In [23]:
zf = zipfile.ZipFile('./cda_fi.zip') 

# Patrimônio Líquido
cda0 = pd.read_csv(zf.open('cda_fi_PL' + comptc() + '.csv'),    sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'VL_PATRIM_LIQ'], dtype=object)             
                          
# 1. Titulos Públicos
cda1 = pd.read_csv(zf.open('cda_fi_BLC_1' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'VL_MERC_POS_FINAL', 'CD_ISIN', 'TP_TITPUB', 'DT_VENC'], 
                   dtype=object)\
                   [['CNPJ_FUNDO','TP_APLIC', 'VL_MERC_POS_FINAL','CD_ISIN', 'TP_TITPUB','DT_VENC']].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC','TP_TITPUB': 'DS_ATIVO'})\
                                
# 2. Cotas de Fundos
cda2 = pd.read_csv(zf.open('cda_fi_BLC_2' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'VL_MERC_POS_FINAL', 'CNPJ_FUNDO_COTA'], 
                   dtype=object)\
                   [['CNPJ_FUNDO', 'TP_APLIC', 'VL_MERC_POS_FINAL', 'CNPJ_FUNDO_COTA']].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC'})     

# 3. SWAP
cda3 = pd.read_csv(zf.open('cda_fi_BLC_3' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'VL_MERC_POS_FINAL', 'CD_SWAP', 'DS_SWAP'], 
                   dtype=object)\
                   [['CNPJ_FUNDO', 'TP_APLIC', 'VL_MERC_POS_FINAL', 'CD_SWAP', 'DS_SWAP']].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC','DS_SWAP': 'CD_ATIVO'})  

# 4. Ativos Codificados
cda4 = pd.read_csv(zf.open('cda_fi_BLC_4' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'CD_ISIN', 'CD_ATIVO' ], 
                   dtype=object)\
                   [['CNPJ_FUNDO', 'TP_APLIC', 'VL_MERC_POS_FINAL', 'CD_ISIN', 'CD_ATIVO', 'TP_ATIVO' ]].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC','TP_ATIVO': 'DS_ATIVO'})                        

# 5. Depósitos a Prazo e Outros Títulos de IF
cda5 = pd.read_csv(zf.open('cda_fi_BLC_5' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DT_VENC', 'CNPJ_EMISSOR', 'EMISSOR'], 
                   dtype=object)\
                   [['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DT_VENC', 'CNPJ_EMISSOR', 'EMISSOR']].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC', 'CNPJ_EMISSOR': 'CPF_CNPJ_EMISSOR'})       

# 6. Títulos do Agronegócio e de Crédito Privado
cda6 = pd.read_csv(zf.open('cda_fi_BLC_6' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DT_VENC', 'CPF_CNPJ_EMISSOR', 'EMISSOR'], 
                   dtype=object)\
                   [['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DT_VENC', 'CPF_CNPJ_EMISSOR', 'EMISSOR']].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC'})     

# 7. Investimento no Exterior
cda7 = pd.read_csv(zf.open('cda_fi_BLC_7' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'CD_ATIVO_BV_MERC', 'BV_MERC', 'PAIS', 'DT_VENC', 'EMISSOR'], 
                   dtype=object)\
                   [['CNPJ_FUNDO', 'TP_APLIC', 'VL_MERC_POS_FINAL', 'CD_ATIVO_BV_MERC', 'BV_MERC', 'PAIS', 'TP_ATIVO', 'DT_VENC', 'EMISSOR']].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC','CD_ATIVO_BV_MERC': 'CD_ATIVO'})      

# 8. Ativos não Codificados
cda8 = pd.read_csv(zf.open('cda_fi_BLC_8' + comptc() + '.csv'), sep=';', encoding='ISO-8859-1', 
                   usecols=['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DS_ATIVO', 'CPF_CNPJ_EMISSOR', 'EMISSOR'], 
                   dtype=object)\
                   [['CNPJ_FUNDO', 'TP_APLIC', 'TP_ATIVO', 'VL_MERC_POS_FINAL', 'DS_ATIVO', 'CPF_CNPJ_EMISSOR', 'EMISSOR']].\
                   rename(columns={'VL_MERC_POS_FINAL': 'VL_APLIC'})             

In [24]:
cda5['DS_ATIVO'] = cda5.apply(lambda row: aplic_ativo(row), axis=1)
cda6['DS_ATIVO'] = cda6.apply(lambda row: aplic_ativo(row), axis=1)
cda7['DS_ATIVO'] = cda7.apply(lambda row: aplic_ativo(row), axis=1)
cda8['DS_ATIVO'] = cda8.apply(lambda row: aplic_ativo_descricao(row), axis=1)

cda5 = cda5.drop(['TP_ATIVO'], axis=1)
cda6 = cda6.drop(['TP_ATIVO'], axis=1)
cda7 = cda7.drop(['TP_ATIVO'], axis=1)
cda8 = cda8.drop(['TP_ATIVO'], axis=1)

### Datasets de ativos e emissores (Site B3)

In [25]:
zf = zipfile.ZipFile('./isinp.zip') 
ativos_isin = pd.read_csv(zf.open('NUMERACA.TXT'), sep=',', header=None, \
                   usecols=[2, 3, 5, 20], dtype=object, \
                   names=['CD_ISIN', 'COD_EMISSOR', 'DS_ATIVO', 'COD_TIPO'])\
                   [['CD_ISIN', 'COD_TIPO', 'DS_ATIVO', 'COD_EMISSOR']]
emissores_ativos_isin = pd.read_csv(zf.open('EMISSOR.TXT') , sep=',', header=None, \
                   usecols=[0, 1, 2], dtype=object, \
                   names=['COD_EMISSOR', 'EMISSOR', 'CNPJ_EMISSOR'])

In [26]:
emissores_ativos_isin['CNPJ_EMISSOR'] = emissores_ativos_isin.CNPJ_EMISSOR.fillna('').apply(formataCNPJ)
emissores_ativos_isin = emissores_ativos_isin.rename(columns={'CNPJ_EMISSOR': 'CPF_CNPJ_EMISSOR'})

## Tratamento dos dados

### Fundos


In [27]:
# Inspecionando domínio do campo SIT (Situação do fundo)
print(cad_fundo.SIT.unique())

['CANCELADA' 'EM FUNCIONAMENTO NORMAL' 'LIQUIDAÇÃO' 'INCORPORAÇÃO'
 'FASE PRÉ-OPERACIONAL' 'EM SITUAÇÃO ESPECIAL' 'TRANSFORMAÇÃO'
 'EM ANÁLISE']


In [28]:
# Inspecionando fundos com mais de uma linha no dataset
cad_fundo[cad_fundo.duplicated(subset=['CNPJ_FUNDO'], keep=False)].sort_values('CNPJ_FUNDO')

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,DT_INI_SIT,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR
5278,FI,00.000.432/0001-00,COPESUL - FUNDO DE INVESTIMENTO MULTIMERCADO ...,2005-03-31,CANCELADA,2008-07-18,False,True,NaN
49560,FIF,00.000.432/0001-00,COPESUL - FUNDO DE INVESTIMENTO MULTIMERCADO ...,2003-04-30,CANCELADA,2005-03-31,NaN,NaN,NaN
49566,FIF,00.000.744/0001-06,REAL FUNDO DE INVESTIMENTO CURTO PRAZO DAILY A,2003-04-30,CANCELADA,2005-01-01,NaN,NaN,NaN
5279,FI,00.000.744/0001-06,REAL FUNDO DE INVESTIMENTO CURTO PRAZO DAILY A,2005-01-13,CANCELADA,2006-06-30,False,False,NaN
5280,FI,00.000.746/0001-03,ALFA DINÂMICO - FUNDO DE INVESTIMENTO EM COTAS...,2004-12-09,CANCELADA,2016-09-26,True,False,60.770.336/0001-65
...,...,...,...,...,...,...,...,...,...
49348,FIDC,97.525.450/0001-30,G5 AP INFRA FUNDO DE INVESTIMENTO EM DIREITOS ...,2017-12-07,EM FUNCIONAMENTO NORMAL,2017-12-07,False,NaN,09.446.129/0001-00
46482,FI,97.525.450/0001-30,G5 AP INFRA FUNDO DE INVESTIMENTO EM DIREITOS ...,2011-08-10,CANCELADA,2017-12-07,False,False,NaN
46483,FI,97.525.452/0001-29,G5 SCB II FUNDO DE INVESTIMENTO EM AÇÕES,2011-08-10,CANCELADA,2017-12-19,False,False,NaN
46484,FI,97.525.452/0001-29,G5 SCB II FUNDO DE INVESTIMENTO EM AÇÕES,2020-02-12,EM FUNCIONAMENTO NORMAL,2020-02-12,False,False,09.446.129/0001-00


In [29]:
# Selecionando as linhas correspondentes à situação mais recente do fundo
fundos_ultima_situacao = cad_fundo[['TP_FUNDO','CNPJ_FUNDO','DENOM_SOCIAL','DT_REG', 'SIT','DT_INI_SIT','FUNDO_COTAS','FUNDO_EXCLUSIVO']]
fundos_ultima_situacao['SIT_AUX'] = fundos_ultima_situacao.apply(lambda row: sit_aux(row), axis=1)

fundos_ultima_situacao = fundos_ultima_situacao.sort_values(by=['DT_INI_SIT', 'DT_REG', 'SIT_AUX']).drop_duplicates(subset=['CNPJ_FUNDO'],keep='last')
fundos_ultima_situacao = fundos_ultima_situacao.drop(['DT_REG'], axis=1)
fundos_ultima_situacao = fundos_ultima_situacao.drop(['SIT_AUX'], axis=1)

fundos_ultima_situacao

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,SIT,DT_INI_SIT,FUNDO_COTAS,FUNDO_EXCLUSIVO
59026,FMIA,29.183.837/0001-03,F LAN?A DE A,CANCELADA,1986-05-25,NaN,NaN
59124,FMIA,55.221.998/0001-46,F PRICE DE A,CANCELADA,1987-06-05,NaN,NaN
59134,FMIA,56.093.081/0001-76,F ULTRACRED DE A,CANCELADA,1987-09-03,NaN,NaN
59032,FMIA,29.410.727/0001-29,PORTINVEST FIA PORTAL,CANCELADA,1987-10-30,NaN,NaN
59037,FMIA,29.415.700/0001-29,BONANÇA F DE A,CANCELADA,1987-11-10,NaN,NaN
...,...,...,...,...,...,...,...
58005,FIP,41.082.921/0001-28,FUNDO DE INVESTIMENTO EM PARTICIPAÇÕES BTG PAC...,FASE PRÉ-OPERACIONAL,2021-04-01,False,NaN
58008,FIP,41.256.932/0001-87,ROXO ESCARLATE FUNDO DE INVESTIMENTO EM PARTIC...,FASE PRÉ-OPERACIONAL,2021-04-01,False,NaN
49005,FIDC,35.099.744/0001-90,NOVO MUNDO FUNDO DE INVESTIMENTO EM DIREITOS C...,EM ANÁLISE,2021-04-01,False,NaN
35847,FI,29.300.737/0001-01,MANAGER LOGOS FUNDO DE INVESTIMENTO EM COTAS D...,EM FUNCIONAMENTO NORMAL,2021-10-01,True,False


In [30]:
# Inspecionando fundos com mais de uma linha no dataset
fundos_ultima_situacao[fundos_ultima_situacao.duplicated(subset=['CNPJ_FUNDO'], keep=False)].sort_values('CNPJ_FUNDO')

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,SIT,DT_INI_SIT,FUNDO_COTAS,FUNDO_EXCLUSIVO


In [31]:
# Fundos que informaram carteira de aplicações
fundos_aplicacao = cda0[['CNPJ_FUNDO', 'VL_PATRIM_LIQ']]
fundos_aplicacao

,CNPJ_FUNDO,VL_PATRIM_LIQ
0,06.537.068/0001-90,1325.12
1,07.408.147/0001-64,28308.60
2,02.010.153/0001-45,17079539.69
3,02.177.812/0001-32,66438968.44
4,02.177.815/0001-76,2044493.92
...,...,...
19141,04.573.850/0001-30,1531828.38
19142,04.602.186/0001-00,745123.56
19143,04.609.196/0001-77,330930.45
19144,04.609.212/0001-21,1686446.84


In [32]:
# Fundos cujas cotas são ativos em que outros fundos aplicam
fundos_alvo_aplicacao = cda2[['CNPJ_FUNDO_COTA']].drop_duplicates()
fundos_alvo_aplicacao

,CNPJ_FUNDO_COTA
0,08.046.417/0001-05
1,01.608.573/0001-65
2,01.235.622/0001-61
3,10.597.218/0001-29
4,08.277.553/0001-06
...,...
140169,14.038.410/0001-36
140215,32.073.244/0001-90
140349,35.645.145/0001-24
140383,37.236.082/0001-88


In [33]:
# Todos os fundos referenciados no CDA
fundos_cda = pd.concat([fundos_aplicacao,fundos_alvo_aplicacao.rename(columns={'CNPJ_FUNDO_COTA': 'CNPJ_FUNDO'})]).fillna('').sort_values('VL_PATRIM_LIQ').drop_duplicates(subset=['CNPJ_FUNDO'], keep='last')
fundos_cda

,CNPJ_FUNDO,VL_PATRIM_LIQ
14062,13.328.452/0001-49,
13900,18.289.950/0001-43,
14063,16.435.858/0001-19,
13924,11.326.746/0001-06,
13926,31.933.158/0001-48,
...,...,...
16166,35.002.455/0001-20,99971262.26
18755,38.394.157/0001-11,9997326.63
16988,35.844.962/0001-01,99980155.79
17524,36.498.260/0001-86,999891.20


In [34]:
# Seleciona apenas dos fundos referenciados no CDA
fundos = pd.merge(fundos_ultima_situacao, fundos_cda, how="inner", on=["CNPJ_FUNDO"]).fillna('')
fundos

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,SIT,DT_INI_SIT,FUNDO_COTAS,FUNDO_EXCLUSIVO,VL_PATRIM_LIQ
0,FAPI,02.177.812/0001-32,ITAU FAPI RENDA FIXA - FUNDO DE APOSENTADORIA ...,EM FUNCIONAMENTO NORMAL,1997-11-27,,,66438968.44
1,FAPI,02.226.122/0001-26,UNIBANCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,EM FUNCIONAMENTO NORMAL,1997-12-05,,,4740.45
2,FAPI,02.177.815/0001-76,ITAU FAPI CONSERVADOR - FUNDO DE APOSENTADORIA...,EM FUNCIONAMENTO NORMAL,1997-12-16,,,2044493.92
3,FAPI,02.185.027/0001-21,BRADESCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,EM FUNCIONAMENTO NORMAL,1997-12-19,,,453086833.64
4,FAPI,02.297.533/0001-02,BRADESCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,EM FUNCIONAMENTO NORMAL,1997-12-29,,,15654242.81
...,...,...,...,...,...,...,...,...
20559,FI,33.667.778/0001-08,CA INDOSUEZ DEBÊNTURES INCENTIVADAS 60 CRÉDITO...,CANCELADA,2021-03-26,True,False,4230106.15
20560,FIP,35.640.839/0001-79,G5 MOBILE II FUNDO DE INVESTIMENTO EM PARTICIP...,LIQUIDAÇÃO,2021-03-29,False,,
20561,FIP,26.314.464/0001-66,BTG PACTUAL INFRAESTRUTURA II ENERGIA FUNDO DE...,LIQUIDAÇÃO,2021-03-30,False,,
20562,FI,29.994.524/0001-27,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,CANCELADA,2021-04-01,True,True,7129047.57


### Gestores

In [35]:
# Fundos e seus gestores
df = cad_fundo[['TP_FUNDO','CNPJ_FUNDO','SIT','DT_INI_SIT','CPF_CNPJ_GESTOR']].dropna().drop_duplicates()

# Associando novamente os gestores aos fundos selecionados
fundos_gestores = pd.merge(fundos, df, how="inner", on=["TP_FUNDO","CNPJ_FUNDO","SIT","DT_INI_SIT"])
fundos_gestores = fundos_gestores[['CNPJ_FUNDO','CPF_CNPJ_GESTOR']]
fundos_gestores

,CNPJ_FUNDO,CPF_CNPJ_GESTOR
0,03.915.495/0001-77,60.701.190/0001-04
1,02.838.578/0001-47,30.822.936/0001-69
2,02.998.275/0001-91,62.375.134/0001-44
3,03.555.959/0001-81,00.360.305/0001-04
4,03.859.784/0001-04,33.311.713/0001-25
...,...,...
21037,17.892.709/0001-41,12.695.840/0001-03
21038,33.667.778/0001-08,01.638.542/0001-57
21039,35.640.839/0001-79,14.121.173/0001-72
21040,26.314.464/0001-66,09.631.542/0001-37


In [36]:
# Inspecionando fundos com mais de uma linha no dataset (mais de um gestor)
fundos_gestores[fundos_gestores.duplicated(subset=['CNPJ_FUNDO'], keep=False)].sort_values('CNPJ_FUNDO')

,CNPJ_FUNDO,CPF_CNPJ_GESTOR
1580,01.392.020/0001-18,62.375.134/0001-44
1579,01.392.020/0001-18,51.990.695/0001-37
1532,01.392.021/0001-62,51.990.695/0001-37
1533,01.392.021/0001-62,62.375.134/0001-44
1582,01.606.509/0001-45,62.375.134/0001-44
...,...,...
19572,37.828.367/0001-08,12.330.774/0001-60
20766,37.900.216/0001-13,16.789.525/0001-98
20767,37.900.216/0001-13,29.408.732/0001-05
20348,38.489.304/0001-37,18.506.057/0001-22


In [37]:
cad_gestor = cad_gestor.dropna().drop_duplicates()

gestores = pd.merge(cad_gestor, fundos_gestores, how="inner", on=["CPF_CNPJ_GESTOR"])
gestores = gestores[['CPF_CNPJ_GESTOR','GESTOR']].dropna().drop_duplicates(subset='CPF_CNPJ_GESTOR', keep="last")

gestores

,CPF_CNPJ_GESTOR,GESTOR
60,60.770.336/0001-65,BANCO ALFA DE INVESTIMENTO S.A.
1070,60.701.190/0001-04,ITAU UNIBANCO S.A.
1095,33.172.537/0001-98,BANCO J.P. MORGAN S.A.
1707,12.695.840/0001-03,JULIUS BAER FAMILY OFFICE BRASIL GESTÃO DE PAT...
3277,62.375.134/0001-44,BRAM - BRADESCO ASSET MANAGEMENT S.A. DISTRIBU...
...,...,...
22482,09.544.103/0001-97,INVESTIMAGE ADMINISTRADORA DE RECURSOS LTDA.
22484,29.292.940/0001-83,REDPOINT EVENTURES GESTÃO DE RECURSOS LTDA.
22486,35.096.963/0001-15,KAMAROOPIN GESTORA DE RECURSOS LTDA.
22488,35.098.686/0001-80,10B GESTORA DE RECURSOS LTDA.


### Emissores

In [38]:
emissores_cda5  = cda5[['CPF_CNPJ_EMISSOR','EMISSOR']].dropna(how='all').drop_duplicates().fillna('')
emissores_cda6  = cda6[['CPF_CNPJ_EMISSOR','EMISSOR']].dropna(how='all').drop_duplicates().fillna('') 
emissores_cda7  = cda7[['EMISSOR']].dropna().drop_duplicates()
emissores_cda8  = cda8[['CPF_CNPJ_EMISSOR','EMISSOR']].dropna(how='all').drop_duplicates().fillna('')

In [39]:
# Verifica se não há valores ausentes ('missing values') em CPF_CNPJ_EMISSOR 
df = emissores_cda5[emissores_cda5['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


In [40]:
# Verifica se não há valores ausentes ('missing values') em CPF_CNPJ_EMISSOR 
df = emissores_cda6[emissores_cda6['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


In [41]:
# Verifica se não há valores ausentes ('missing values') em CPF_CNPJ_EMISSOR 
df = emissores_cda8[emissores_cda8['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

False


In [42]:
# Emissores com CNPJ - Tabela ISIN (B3)
emissores_isin_cnpj = emissores_ativos_isin[emissores_ativos_isin['CPF_CNPJ_EMISSOR']!=''][['CPF_CNPJ_EMISSOR','EMISSOR']]

# Emissores com CNPJ - Aplicações bloco 8 cda (CVM)
emissores_cda8_cnpj = emissores_cda8[emissores_cda8['CPF_CNPJ_EMISSOR']!=''][['CPF_CNPJ_EMISSOR','EMISSOR']]

# Emissores sem CNPJ - Tabela ISIN (B3)
emissores_isin_sem_cnpj  = emissores_ativos_isin[emissores_ativos_isin['CPF_CNPJ_EMISSOR']==''][['EMISSOR']]

# Emissores sem CNPJ - Aplicações bloco 8 cda (CVM)
emissores_cda8_sem_cnpj  = emissores_cda8[emissores_cda8['CPF_CNPJ_EMISSOR']==''][['EMISSOR']]

In [43]:
emissores_isin_cnpj
emissores_isin_sem_cnpj

,EMISSOR
659,BB CONCESSÕES BRASIL 3 FUNDO DE INVESTI MENTO ...
1687,CAPITÂNIA PREV ADVISORY XP SEGUROS FUNDO DE IN...
2149,LEGACY CAPITAL FUNDO DE INVESTIMENTO EM COTAS ...
9985,CARTEIRA CORPORATIVA 191 FUNDO INVEST COTAS FU...
10101,ADVANCE AUTO PARTS INC
...,...
43057,BCO EUROPEU AMERICA LATINA SA
43058,FRANLEASE ARREND MERCANTIL SA
43059,CIA BRAS PROJETOS OBRAS
43060,BASF BRASILEIRA SA INDS QUIMICAS


In [44]:
emissores_cda8_cnpj
emissores_cda8_sem_cnpj

,EMISSOR
6012,INDICE BOVESPA
10456,Ishares Msci All Country Asia Ex Japan Etf Bdr
10458,Ishares Msci Emerging Markets Etf Bdr
10459,Ishares Msci Eafe Etf Bdr
10460,Ishares Russel 2000 Etf Bdr
...,...
160339,Participacao/TROPICAL
160382,Outros/REC SAPHYR IPIRANGA
160439,Participacao/VHR EMPREENDIMENTOS
160470,Acoes Judiciais/CCE BMG - BOM JESUS


In [45]:
# Verifica se não há valores ausentes ('missing values') em CPF_CNPJ_EMISSOR 
df = emissores_cda8_cnpj[emissores_cda8_cnpj['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


In [46]:
# Verifica se não há valores ausentes ('missing values') em CPF_CNPJ_EMISSOR 
df = emissores_isin_cnpj[emissores_isin_cnpj['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


In [47]:
nomes_emissores_cnpj = pd.concat([emissores_cda5, emissores_cda6, emissores_cda8_cnpj, emissores_isin_cnpj])

nomes_emissores_cnpj = nomes_emissores_cnpj.drop_duplicates(subset='CPF_CNPJ_EMISSOR', keep="last")

nomes_emissores_cnpj

,CPF_CNPJ_EMISSOR,EMISSOR
2631,07.810.517/0001-95,BANCO BTG PACTUAL S/A
2724,18.236.120/0001-58,NU PAGAMENTOS
5056,30.121.792/0001-13,CREDIT SUISSE
8326,60.701.190/4816-09,ITAU UNIBANCO S
8771,11.476.673/0001-39,BANCO RANDON SA
...,...,...
43054,35.993.098/0001-00,Z.A DIGITAL DE SÃO PAULO SISTEMA DE ESTACIONAM...
43055,26.470.438/0001-27,JBZ FUNDO INVESTIMENTO COTAS FUNDOS INVESTIMEN...
43056,24.290.332/0001-16,SANTANDER PB ZARAGOZA MULTIMERCADO CRÉDITO PRI...
43061,23.637.697/0001-01,ALCOA ALUMINIO SA


In [48]:
# Verifica se não há valores ausentes ('missing values') em EMISSOR (Nome do Emissor) 
df = emissores_cda7[emissores_cda7['EMISSOR']=='']
print(df.empty) 

True


In [49]:
# Verifica se não há valores ausentes ('missing values') em EMISSOR (Nome do Emissor) 
df = emissores_cda8_sem_cnpj[emissores_cda8_sem_cnpj['EMISSOR']=='']
print(df.empty) 

True


In [50]:
# Verifica se não há valores ausentes ('missing values') em EMISSOR (Nome do Emissor) 
df = emissores_isin_sem_cnpj[emissores_isin_sem_cnpj['EMISSOR']=='']
print(df.empty) 

True


In [51]:
nomes_emissores_sem_cnpj = pd.concat([emissores_cda7, emissores_cda8_sem_cnpj, emissores_isin_sem_cnpj])

nomes_emissores_sem_cnpj = nomes_emissores_sem_cnpj.drop_duplicates()

nomes_emissores_sem_cnpj.head()

,EMISSOR
0,TEMPUS OPP 1 CLASS
1,ALIBABA GROUP HOLDING LTD
2,AMAZON.COM INC
3,JP MORGAN CHASE CO.
4,GOVERNO BRASILEIRO


In [52]:
nomes_emissores = pd.concat([nomes_emissores_cnpj, nomes_emissores_sem_cnpj])

nomes_emissores.reset_index(drop=True, inplace=True)

nomes_emissores

,CPF_CNPJ_EMISSOR,EMISSOR
0,07.810.517/0001-95,BANCO BTG PACTUAL S/A
1,18.236.120/0001-58,NU PAGAMENTOS
2,30.121.792/0001-13,CREDIT SUISSE
3,60.701.190/4816-09,ITAU UNIBANCO S
4,11.476.673/0001-39,BANCO RANDON SA
...,...,...
45467,NaN,BCO EUROPEU AMERICA LATINA SA
45468,NaN,FRANLEASE ARREND MERCANTIL SA
45469,NaN,CIA BRAS PROJETOS OBRAS
45470,NaN,BASF BRASILEIRA SA INDS QUIMICAS


In [53]:
# Verifica unicidade de CPF_CNPJ
cnt = nomes_emissores[nomes_emissores['CPF_CNPJ_EMISSOR']==''].groupby('CPF_CNPJ_EMISSOR')['EMISSOR'].nunique()
print(cnt[cnt>1].empty)

True


In [54]:
# Verifica se não há valores ausentes ('missing values') em EMISSOR (Nome do Emissor) 
df = nomes_emissores[nomes_emissores['EMISSOR']=='']
print(df.empty) 

True


In [55]:
# Verifica duplicidade em EMISSOR (Nome do Emissor)
cnt = nomes_emissores.groupby('EMISSOR')['CPF_CNPJ_EMISSOR'].nunique()
cnt[cnt>1].count()
#cnt[cnt>1]
#nomes_emissores[nomes_emissores['EMISSOR']=='ODEBRECHT SA']

46

### Ativos


In [56]:
ativos_isin

,CD_ISIN,COD_TIPO,DS_ATIVO,COD_EMISSOR
0,BR0002CTF008,CTF,COTAS SENIORES,0002
1,BR0002CTF016,CTF,COTAS SUBORDINADAS,0002
2,BR0003CTF006,CTF,COTAS,0003
3,BR0004CTF004,CTF,COTAS,0004
4,BR0005CTF001,CTF,COTAS,0005
...,...,...,...,...
126517,BRZZDTDBS005,DBS,DEBENTURES SIMPLES 1ª EMISSÃO SÉRIE 1ª,ZZDT
126518,BRZZDTDBS013,DBS,DEBENTURES SIMPLES 1ª EMISSÃO SÉRIE 2ª,ZZDT
126519,BRZZF1CTF007,CTF,COTAS COTAS,ZZF1
126520,BRZZGRCTF003,CTF,COTAS COTAS,ZZGR


In [57]:
titulos_publicos        = cda1[['CD_ISIN','DS_ATIVO','DT_VENC']].drop_duplicates().fillna('') 
swap                    = cda3[['CD_SWAP','CD_ATIVO']].drop_duplicates().fillna('') 
ativos_codificados      = cda4[['CD_ISIN','CD_ATIVO','DS_ATIVO']].drop_duplicates().fillna('') 
depositos_prazo         = cda5[['DS_ATIVO','DT_VENC','CPF_CNPJ_EMISSOR']].drop_duplicates().fillna('')
titulos_agronegocio     = cda6[['DS_ATIVO','DT_VENC','CPF_CNPJ_EMISSOR']].drop_duplicates().fillna('') 
investimento_exterior   = cda7[['CD_ATIVO','BV_MERC','PAIS','DS_ATIVO','DT_VENC','EMISSOR']].drop_duplicates().fillna('') 
ativos_nao_codificados  = cda8[['DS_ATIVO','CPF_CNPJ_EMISSOR']].drop_duplicates().fillna('')

#### Títulos Públicos

In [58]:
df1 = pd.merge(titulos_publicos.rename(columns={'DS_ATIVO': 'DS_ATIVO_1'}), ativos_isin.rename(columns={'DS_ATIVO': 'DS_ATIVO_2'}),
                        how="left", on=["CD_ISIN"])

df1 = df1.replace(np.nan, '', regex=True)

df1['DS_ATIVO'] = df1.apply(lambda row: ds_ativo(row), axis=1)

df1 = df1.drop(['COD_TIPO'], axis=1)
df1 = df1.drop(['DS_ATIVO_1'], axis=1)
df1 = df1.drop(['DS_ATIVO_2'], axis=1)

df2 = pd.merge(df1, emissores_ativos_isin, 
               how = "left", on=["COD_EMISSOR"])

# A coluna CPF_CNPJ_EMISSOR do dataframe 'nomes_emissores' será utilizada na operação de merge; por isso, os valores NaN serão preenchidos com '0'
titulos_publicos_emissor = pd.merge(df2.rename(columns={'EMISSOR': 'EMISSOR_1'}), nomes_emissores.rename(columns={'EMISSOR': 'EMISSOR_2'}).fillna({'CPF_CNPJ_EMISSOR': '0'}),
                                    how="left", on=["CPF_CNPJ_EMISSOR"])

titulos_publicos_emissor = titulos_publicos_emissor.drop_duplicates()
titulos_publicos_emissor = titulos_publicos_emissor.replace(np.nan, '', regex=True)

titulos_publicos_emissor['EMISSOR'] = titulos_publicos_emissor.apply(lambda row: emissor(row), axis=1)
titulos_publicos_emissor = titulos_publicos_emissor.drop(['COD_EMISSOR'], axis=1)
titulos_publicos_emissor = titulos_publicos_emissor.drop(['EMISSOR_1'], axis=1)
titulos_publicos_emissor = titulos_publicos_emissor.drop(['EMISSOR_2'], axis=1)

titulos_publicos_emissor

,CD_ISIN,DT_VENC,DS_ATIVO,CPF_CNPJ_EMISSOR,EMISSOR
0,BRSTNCLF1R74,2022-09-01,LETRAS FINANCEIRAS DO TESOURO LETRAS FINANCEIR...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
1,BRSTNCNTF1P8,2027-01-01,NOTAS DO TESOURO NACIONAL SERIE F NOTAS DO TES...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
2,BRSTNCLTN7O0,2022-07-01,LETRAS DO TESOURO NACIONAL LETRAS DO TESOURO N...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
3,BRSTNCNTB4O9,2023-05-15,NOTAS DO TESOURO NACIONAL SERIE B NOTAS DO TES...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
4,BRSTNCNTB096,2024-08-15,NOTAS DO TESOURO NACIONAL SERIE B NOTAS DO TES...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
...,...,...,...,...,...
111,BRSTNCNI17E6,2025-09-15,NOTAS DO TESOURO NACIONAL SERIE I NOTAS DO TES...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
112,BRSTNCNI1860,2025-10-15,NOTAS DO TESOURO NACIONAL SERIE I NOTAS DO TES...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
113,BRSTNCNI18W6,2026-02-15,NOTAS DO TESOURO NACIONAL SERIE I NOTAS DO TES...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL
114,BRSTNCNI1878,2026-04-15,NOTAS DO TESOURO NACIONAL SERIE I NOTAS DO TES...,00.394.460/0409-50,SECRETARIA TESOURO NACIONAL


#### SWAP

In [59]:
df1 = pd.merge(swap.rename(columns={'CD_SWAP': 'COD_TIPO'}), ativos_isin,
                        how="left", on=["COD_TIPO"])

df1 = df1.replace(np.nan, '', regex=True)

df1 = df1.drop(['COD_TIPO'], axis=1)

df2 = pd.merge(df1, emissores_ativos_isin, 
              how = "left", on=["COD_EMISSOR"])

# A coluna CPF_CNPJ_EMISSOR do dataframe 'nomes_emissores' será utilizada na operação de merge; por isso, os valores NaN serão preenchidos com '0'
swap_emissor = pd.merge(df2.rename(columns={'EMISSOR': 'EMISSOR_1'}), nomes_emissores.rename(columns={'EMISSOR': 'EMISSOR_2'}).fillna({'CPF_CNPJ_EMISSOR': '0'}),
                                    how="left", on=["CPF_CNPJ_EMISSOR"])

swap_emissor = swap_emissor.drop_duplicates()
swap_emissor = swap_emissor.replace(np.nan, '', regex=True)

swap_emissor['EMISSOR'] = swap_emissor.apply(lambda row: emissor(row), axis=1)
swap_emissor = swap_emissor.drop(['COD_EMISSOR'], axis=1)
swap_emissor = swap_emissor.drop(['EMISSOR_1'], axis=1)
swap_emissor = swap_emissor.drop(['EMISSOR_2'], axis=1)

swap_emissor

,CD_ATIVO,CD_ISIN,DS_ATIVO,CPF_CNPJ_EMISSOR,EMISSOR
0,DI1 X IAP,BRBMEFSDL000,SWAP - TAXA DI DE UM DIA VERSUS IPCA/IBGE SWAP...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
1,DI1 X PRE,BRBMEFSDP001,SWAP - TAXA DI DE UM DIA VERSUS TAXA PREFIXADA...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
2,TBF X PRE,BRBMEFSBP005,SWAP - TBF VERSUS TAXA PREFIXADA SWAP - TBF VE...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
3,DI1 X DOL,BRBMEFSDC009,SWAP - TAXA DI DE UM DIA VERSUS DOLAR COMERCI...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
4,SB1 X DI1,BRBMEFSBD001,SWAP - CARTEIRA DE ACOES VERSUS TAXA DI DE UM ...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
5,DOL X PRE,BRBMEFSCP003,SWAP - DOLAR COMERCIAL VERSUS TAXA PREFIXADA S...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
6,PRE X SB1,BRBMEFSPB005,SWAP - TAXA PREFIXADA VERSUS CARTEIRA DE ACOES...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
7,DI1 X REU,BRBMEFSDE005,SWAP - TAXA DI DE UM DIA VERSUS EURO SWAP - TA...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
8,REU X PRE,BRBMEFSEP009,SWAP - EURO VERSUS TAXA PREFIXADA SWAP - EURO ...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
9,DI1 X IGM,BRBMEFSDM008,SWAP - TAXA DI DE UM DIA VERSUS IGP-M SWAP - T...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA


#### Ativos Codificados

In [60]:
df1 = pd.merge(ativos_codificados.rename(columns={'DS_ATIVO': 'DS_ATIVO_1'}), ativos_isin.rename(columns={'DS_ATIVO': 'DS_ATIVO_2'}),
                        how="left", on=["CD_ISIN"])

df1 = df1.replace(np.nan, '', regex=True)

df1['DS_ATIVO'] = df1.apply(lambda row: ds_ativo(row), axis=1)

df1 = df1.drop(['COD_TIPO'], axis=1)
df1 = df1.drop(['DS_ATIVO_1'], axis=1)
df1 = df1.drop(['DS_ATIVO_2'], axis=1)

df2 = pd.merge(df1, emissores_ativos_isin, 
               how = "left", on=["COD_EMISSOR"])

# A coluna CPF_CNPJ_EMISSOR do dataframe 'nomes_emissores' será utilizada na operação de merge; por isso, os valores NaN serão preenchidos com '0'
ativos_codificados_emissor = pd.merge(df2.rename(columns={'EMISSOR': 'EMISSOR_1'}), nomes_emissores.rename(columns={'EMISSOR': 'EMISSOR_2'}).fillna({'CPF_CNPJ_EMISSOR': '0'}),
                                    how="left", on=["CPF_CNPJ_EMISSOR"])

ativos_codificados_emissor = ativos_codificados_emissor.drop_duplicates()
ativos_codificados_emissor = ativos_codificados_emissor.replace(np.nan, '', regex=True)

ativos_codificados_emissor['EMISSOR'] = ativos_codificados_emissor.apply(lambda row: emissor(row), axis=1)
ativos_codificados_emissor = ativos_codificados_emissor.drop(['COD_EMISSOR'], axis=1)
ativos_codificados_emissor = ativos_codificados_emissor.drop(['EMISSOR_1'], axis=1)
ativos_codificados_emissor = ativos_codificados_emissor.drop(['EMISSOR_2'], axis=1)

ativos_codificados_emissor

,CD_ISIN,CD_ATIVO,DS_ATIVO,CPF_CNPJ_EMISSOR,EMISSOR
0,BRUGPADBS067,UGPA16,DEBENTURES SIMPLES DEBENTURES SIMPLES SEXTA EM...,33.256.439/0001-39,ULTRAPAR PARTICIPACOES SA
1,BRVIVTDBS069,VIVT15,DEBENTURES SIMPLES DEBENTURES SIMPLES QUINTA E...,02.558.157/0001-62,TELEFONICA BRASIL SA
2,BRPCARDBS0B7,CBRDA5,Debênture simples,,
3,BRCMGDDBS033,CMDT23,Debênture simples,,
4,BRIGTADBS049,IGTA24,Debênture simples,,
...,...,...,...,...,...
4237,BRS1REBDR009,S1RE34,CERTIFICADO DE DEPOSITO DE VALORES MOBILIARIOS...,,SEMPRA ENERGY
4238,BRS1YKBDR002,S1YK34,CERTIFICADO DE DEPOSITO DE VALORES MOBILIARIOS...,,STRYKER CORP
4239,BRWATCBDR009,WATC34,CERTIFICADO DE DEPOSITO DE VALORES MOBILIARIOS...,,WATERS CORP
4240,BRB1ILBDR002,B1IL34,CERTIFICADO DE DEPOSITO DE VALORES MOBILIARIOS...,,BILIBILI INC


#### Depósitos a Prazo

In [61]:
# Verifica se não há valores ausentes em CPF_CNPJ_EMISSOR 
df = depositos_prazo[depositos_prazo['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


In [62]:
# A coluna CPF_CNPJ_EMISSOR do dataframe 'nomes_emissores' será utilizada na operação de merge; por isso, os valores NaN serão preenchidos com '0'
depositos_prazo_emissor = pd.merge(depositos_prazo, nomes_emissores.fillna({'CPF_CNPJ_EMISSOR': '0'}),
                        how="left", on=["CPF_CNPJ_EMISSOR"])

depositos_prazo_emissor = depositos_prazo_emissor.drop_duplicates()
depositos_prazo_emissor = depositos_prazo_emissor.replace(np.nan, '', regex=True)

depositos_prazo_emissor

,DS_ATIVO,DT_VENC,CPF_CNPJ_EMISSOR,EMISSOR
0,Depósitos a prazo e outros títulos de IF - Let...,2022-07-15,60.701.190/0001-04,BCO ITAU SA
1,Depósitos a prazo e outros títulos de IF - Let...,2022-09-15,60.746.948/0001-12,BCO BRADESCO SA
2,Depósitos a prazo e outros títulos de IF - Let...,2022-10-17,60.746.948/0001-12,BCO BRADESCO SA
3,Depósitos a prazo e outros títulos de IF - Let...,2022-11-16,60.701.190/0001-04,BCO ITAU SA
4,Depósitos a prazo e outros títulos de IF - Let...,2021-03-15,58.160.789/0001-28,BCO SAFRA SA
...,...,...,...,...
5121,Depósitos a prazo e outros títulos de IF - CDB...,2021-11-11,30.306.294/0001-45,BCO BTG PACTUAL SA
5122,Depósitos a prazo e outros títulos de IF - Let...,2022-10-17,90.400.888/0001-42,BCO SANTANDER (BRASIL) SA
5123,Depósitos a prazo e outros títulos de IF - CDB...,2021-10-29,30.306.294/0001-45,BCO BTG PACTUAL SA
5124,Depósitos a prazo e outros títulos de IF - CDB...,2021-11-18,30.306.294/0001-45,BCO BTG PACTUAL SA


#### Títulos do Agronegócio

In [63]:
# Verifica se não há valores ausentes em CPF_CNPJ_EMISSOR 
df = titulos_agronegocio[titulos_agronegocio['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


In [64]:
# A coluna CPF_CNPJ_EMISSOR do dataframe 'nomes_emissores' será utilizada na operação de merge; por isso, os valores NaN serão preenchidos com '0'
titulos_agronegocio_emissor = pd.merge(titulos_agronegocio, nomes_emissores.fillna({'CPF_CNPJ_EMISSOR': '0'}),
                        how="left", on=["CPF_CNPJ_EMISSOR"])

titulos_agronegocio_emissor = titulos_agronegocio_emissor.drop_duplicates()
titulos_agronegocio_emissor = titulos_agronegocio_emissor.replace(np.nan, '', regex=True)

titulos_agronegocio_emissor

,DS_ATIVO,DT_VENC,CPF_CNPJ_EMISSOR,EMISSOR
0,Debêntures - Debênture simples,2024-08-14,09.346.601/0001-25,"B3 SA - BRASIL, BOLSA, BALCAO"
1,Debêntures - Debênture simples,2025-09-15,02.919.555/0001-67,ARTERIS SA
2,Debêntures - Debênture simples,2022-08-15,33.000.167/0001-01,PETROLEO BRASILEIRO SA PETROBRAS
3,Debêntures - Debênture simples,2026-05-06,33.611.500/0001-19,GERDAU SA
4,Debêntures - Debênture simples,2025-11-04,19.276.528/0001-16,BCBF PARTICIPACOES S.A
...,...,...,...,...
1290,Títulos de Crédito Privado - CCB,2022-02-17,45.237.724/0001-91,J MOREIRA COMER
1291,Títulos de Crédito Privado - CCB,2022-12-01,09.584.001/0002-86,LUMINAE S.A.
1292,Debêntures - Debênture simples,2029-09-26,15.009.178/0001-70,USINAS ITAMARATI SA
1293,Debêntures - Debênture simples,2022-12-31,15.009.178/0001-70,USINAS ITAMARATI SA


#### Investimento no Exterior

In [65]:
# Verifica se não há valores ausentes em EMISSOR (Nome do Emissor) 
df = investimento_exterior[investimento_exterior['EMISSOR']=='']
print(df.empty) 

False


In [66]:
investimento_exterior_emissor = investimento_exterior
investimento_exterior_emissor

,CD_ATIVO,BV_MERC,PAIS,DS_ATIVO,DT_VENC,EMISSOR
0,OPPTEMP1,Outras bolsas ou mercados de balcão organizados,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Outros,,TEMPUS OPP 1 CLASS
1,BABA US,NYSE EURONEXT,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Ação ordinária,,ALIBABA GROUP HOLDING LTD
2,AMZN,NASDAQ STOCK MARKET,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Ação ordinária,,AMAZON.COM INC
3,JPMC,NYSE EURONEXT,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Ação ordinária,,JP MORGAN CHASE CO.
4,EJ9011747,Outras bolsas ou mercados de balcão organizados,BRASIL,Investimento no Exterior - Título da Dívida Ex...,2025-01-07,GOVERNO BRASILEIRO
...,...,...,...,...,...,...
16404,5096580,Outras bolsas ou mercados de balcão organizados,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Título da Dívida Ex...,2026-04-07,TESOURO NAC
16408,IBB US,NASDAQ STOCK MARKET,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Depository Receipt ...,,ISHARES NASDAQ BIOTECHNOLOGY
16409,IGV US,Outras bolsas ou mercados de balcão organizados,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Depository Receipt ...,,ISHARES NORTH AMERICAN TECH
16412,OKTA US,NASDAQ STOCK MARKET,ESTADOS UNIDOS DA AMÉRICA,Investimento no Exterior - Depository Receipt ...,,OKTA INC


#### Ativos Não Codificados

In [67]:
# Verifica se não há valores ausentes em CPF_CNPJ_EMISSOR 
df = ativos_nao_codificados[ativos_nao_codificados['CPF_CNPJ_EMISSOR']=='']
print(df.empty)

False


In [68]:
# A coluna CPF_CNPJ_EMISSOR do dataframe 'nomes_emissores' será utilizada na operação de merge; por isso, os valores NaN serão preenchidos com '0'
ativos_nao_codificados_emissor = pd.merge(ativos_nao_codificados, nomes_emissores.fillna({'CPF_CNPJ_EMISSOR': '0'}),
                        how="left", on=["CPF_CNPJ_EMISSOR"])

ativos_nao_codificados_emissor = ativos_nao_codificados_emissor.drop_duplicates()
ativos_nao_codificados_emissor = ativos_nao_codificados_emissor.replace(np.nan, '', regex=True)

ativos_nao_codificados_emissor

,DS_ATIVO,CPF_CNPJ_EMISSOR,EMISSOR
0,Cotas de fundos de renda fixa - Cotas de fundo...,,
1,Disponibilidades - Outros - VALOR DAS DISPONIB...,,
2,Valores a pagar - Outros - VALOR A PAGAR,,
3,Disponibilidades - Outros - BANCO BRASIL CAIXA,00.000.000/0001-91,BCO BRASIL SA
4,Mercado Futuro - Posições compradas - Contrato...,54.641.030/0001-06,BOLSA MERCADORIAS FUTUROS BMF SA
...,...,...,...
49586,Valores a pagar - Outros - DAS AMORT,19.361.060/0001-68,FUNCINE FUNDO FINANC INDUSTRIA CINEMATOGRAFICA...
49587,Valores a pagar - Outros - ELA DISSE,19.361.060/0001-68,FUNCINE FUNDO FINANC INDUSTRIA CINEMATOGRAFICA...
49588,Valores a pagar - Outros - INVESTIMENTO FUNCINE,,
49589,Valores a pagar - Outros - POP,19.361.060/0001-68,FUNCINE FUNDO FINANC INDUSTRIA CINEMATOGRAFICA...


### Aplicações


#### Títulos Públicos

In [69]:
aplic_cda1 = pd.merge(cda1, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda1['PR_PATRIM_LIQ'] = aplic_cda1.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda1 = aplic_cda1.drop(['TP_APLIC'], axis=1)

aplic_cda1 = aplic_cda1.fillna('')
aplic_cda1

,CNPJ_FUNDO,VL_APLIC,CD_ISIN,DS_ATIVO,DT_VENC,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,02.010.153/0001-45,856146.22,BRSTNCLF1R74,LETRAS FINANCEIRAS DO TESOURO,2022-09-01,17079539.69,5.013
1,02.010.153/0001-45,4355691.07,BRSTNCNTF1P8,NOTAS DO TESOURO NACIONAL - SERIE F,2027-01-01,17079539.69,25.502
2,02.010.153/0001-45,7055241.89,BRSTNCLTN7O0,LETRAS DO TESOURO NACIONAL,2022-07-01,17079539.69,41.308
3,02.010.153/0001-45,26512.28,BRSTNCNTB4O9,NOTAS DO TESOURO NACIONAL SERIE B,2023-05-15,17079539.69,0.155
4,02.010.153/0001-45,1265516.96,BRSTNCNTB096,NOTAS DO TESOURO NACIONAL SERIE B,2024-08-15,17079539.69,7.410
...,...,...,...,...,...,...,...
36121,04.573.850/0001-30,343578.84,BRSTNCLF1R58,LETRAS FINANCEIRAS DO TESOURO,2021-09-01,1531828.38,22.429
36122,04.602.186/0001-00,96631.38,BRSTNCLF1R58,LETRAS FINANCEIRAS DO TESOURO,2021-09-01,745123.56,12.969
36123,11.179.262/0001-81,375927.98,BRSTNCLF1R41,LETRAS FINANCEIRAS DO TESOURO,2021-03-01,1780896.83,21.109
36124,11.179.262/0001-81,798183.28,BRSTNCLF1RC4,LETRAS FINANCEIRAS DO TESOURO,2025-03-01,1780896.83,44.819


#### Cotas de Fundos

In [71]:
aplic_cda2 = pd.merge(cda2, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda2['PR_PATRIM_LIQ'] = aplic_cda2.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda2 = aplic_cda2.drop(['TP_APLIC'], axis=1)

aplic_cda2

,CNPJ_FUNDO,VL_APLIC,CNPJ_FUNDO_COTA,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,00.068.305/0001-35,46221487.84,08.046.417/0001-05,46185716.51,100.077
1,00.071.477/0001-68,33160782612.92,01.608.573/0001-65,33158268505.98,100.008
2,00.083.181/0001-67,147100863.98,01.235.622/0001-61,15519640726.95,0.948
3,00.083.181/0001-67,216970194.57,10.597.218/0001-29,15519640726.95,1.398
4,00.083.181/0001-67,1915404901.42,08.277.553/0001-06,15519640726.95,12.342
...,...,...,...,...,...
140422,35.812.806/0001-69,625672.97,32.625.186/0001-60,1746304.69,35.828
140423,35.812.806/0001-69,1095871.64,08.915.927/0001-63,1746304.69,62.754
140424,38.068.651/0001-96,10990110.21,09.577.447/0001-00,14989279.21,73.320
140425,04.604.594/0001-09,537758.38,00.840.011/0001-80,34512288.76,1.558


In [72]:
# Verificar se não há valores ausentes ('missing values') no dataset
print(aplic_cda2.isnull().sum().sum()==0)

True


#### SWAP

In [73]:
aplic_cda3 = pd.merge(cda3, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda3['PR_PATRIM_LIQ'] = aplic_cda3.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda3 = aplic_cda3.drop(['TP_APLIC'], axis=1)
aplic_cda3 = aplic_cda3.drop(['CD_SWAP'], axis=1)

aplic_cda3 = aplic_cda3.fillna('')
aplic_cda3

,CNPJ_FUNDO,VL_APLIC,CD_ATIVO,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,01.622.154/0001-88,6828519.25,DI1 X IAP,580595719.75,-1.176
1,01.630.152/0001-30,894505.58,DI1 X IAP,443673958.28,-0.202
2,02.885.730/0001-42,1692176.12,DI1 X PRE,15070836923.81,-0.011
3,02.885.730/0001-42,26696957.76,DI1 X PRE,15070836923.81,0.177
4,03.191.766/0001-99,604648.44,DI1 X PRE,521985181.54,-0.116
...,...,...,...,...,...
135,71.586.432/0001-74,18201959.17,DI1 X IAP,1659805407.79,-1.097
136,71.586.432/0001-74,5020953.97,DI1 X IAP,1659805407.79,0.303
137,02.727.085/0001-30,0.00,DI1 X PRE,828814.34,0.000
138,03.919.474/0001-20,0.00,DI1 X PRE,2049454.34,0.000


#### Ativos Codificados

In [74]:
aplic_cda4 = pd.merge(cda4, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda4['PR_PATRIM_LIQ'] = aplic_cda4.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda4 = aplic_cda4.drop(['TP_APLIC'], axis=1)

aplic_cda4 = aplic_cda4.fillna('')
aplic_cda4

,CNPJ_FUNDO,VL_APLIC,CD_ISIN,CD_ATIVO,DS_ATIVO,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,02.010.153/0001-45,168985.61,BRUGPADBS067,UGPA16,Debênture simples,17079539.69,0.989
1,02.010.153/0001-45,601192.92,BRVIVTDBS069,VIVT15,Debênture simples,17079539.69,3.520
2,02.010.153/0001-45,175872.88,BRPCARDBS0B7,CBRDA5,Debênture simples,17079539.69,1.030
3,02.010.153/0001-45,53493.45,BRCMGDDBS033,CMDT23,Debênture simples,17079539.69,0.313
4,02.010.153/0001-45,314354.45,BRIGTADBS049,IGTA24,Debênture simples,17079539.69,1.841
...,...,...,...,...,...,...,...
89332,04.609.212/0001-21,12546.60,BRAZULACNPR4,AZUL4,Ação preferencial,1686446.84,0.744
89333,04.609.212/0001-21,6224.68,BRTAEECDAM10,TAEE11,Certificado de depósito de ações,1686446.84,0.369
89334,04.609.212/0001-21,16090.65,BRSANBCDAM13,SANB11,Certificado de depósito de ações,1686446.84,0.954
89335,04.609.212/0001-21,13618.44,BRKLBNCDAM18,KLBN11,Certificado de depósito de ações,1686446.84,0.808


#### Depósitos a Prazo

In [75]:
aplic_cda5 = pd.merge(cda5, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda5['PR_PATRIM_LIQ'] = aplic_cda5.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda5 = aplic_cda5.drop(['TP_APLIC'], axis=1)
aplic_cda5 = aplic_cda5.drop(['EMISSOR'], axis=1)

aplic_cda5 = aplic_cda5.fillna('')
aplic_cda5

,CNPJ_FUNDO,VL_APLIC,DT_VENC,CPF_CNPJ_EMISSOR,DS_ATIVO,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,03.564.825/0001-27,2702144.40,2022-07-15,60.701.190/0001-04,Depósitos a prazo e outros títulos de IF - Let...,91049383.74,2.968
1,03.564.825/0001-27,2556455.90,2022-09-15,60.746.948/0001-12,Depósitos a prazo e outros títulos de IF - Let...,91049383.74,2.808
2,03.564.825/0001-27,1058263.51,2022-10-17,60.746.948/0001-12,Depósitos a prazo e outros títulos de IF - Let...,91049383.74,1.162
3,03.564.825/0001-27,1058263.51,2022-10-17,60.746.948/0001-12,Depósitos a prazo e outros títulos de IF - Let...,91049383.74,1.162
4,03.564.825/0001-27,10353809.66,2022-11-16,60.701.190/0001-04,Depósitos a prazo e outros títulos de IF - Let...,91049383.74,11.372
...,...,...,...,...,...,...,...
26234,97.543.626/0001-86,1661910.14,2025-12-15,10.753.164/0001-43,Depósitos a prazo e outros títulos de IF - CDB...,223204836.49,0.745
26235,97.543.626/0001-86,2008271.57,2022-09-20,58.616.418/0001-08,Depósitos a prazo e outros títulos de IF - DPGE,223204836.49,0.900
26236,97.543.626/0001-86,1006306.14,2022-07-02,61.186.680/0001-74,Depósitos a prazo e outros títulos de IF - DPGE,223204836.49,0.451
26237,97.543.626/0001-86,1547712.97,2026-06-11,58.160.789/0001-28,Depósitos a prazo e outros títulos de IF - Let...,223204836.49,0.693


In [76]:
# Verifica se não há valores ausentes ('missing values') em CPF_CNPJ_EMISSOR 
df = aplic_cda5[aplic_cda5['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


#### Títulos do Agronegócio

In [77]:
aplic_cda6 = pd.merge(cda6, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda6['PR_PATRIM_LIQ'] = aplic_cda6.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda6 = aplic_cda6.drop(['TP_APLIC'], axis=1)
aplic_cda6 = aplic_cda6.drop(['EMISSOR'], axis=1)

aplic_cda6 = aplic_cda6.fillna('')
aplic_cda6

,CNPJ_FUNDO,VL_APLIC,DT_VENC,CPF_CNPJ_EMISSOR,DS_ATIVO,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,02.010.153/0001-45,20310.05,2024-08-14,09.346.601/0001-25,Debêntures - Debênture simples,17079539.69,0.119
1,02.010.153/0001-45,158363.74,2025-09-15,02.919.555/0001-67,Debêntures - Debênture simples,17079539.69,0.927
2,03.564.825/0001-27,2034161.22,2022-08-15,33.000.167/0001-01,Debêntures - Debênture simples,91049383.74,2.234
3,03.564.825/0001-27,2791206.93,2026-05-06,33.611.500/0001-19,Debêntures - Debênture simples,91049383.74,3.066
4,10.546.592/0001-03,2326005.77,2026-05-06,33.611.500/0001-19,Debêntures - Debênture simples,73596008.13,3.161
...,...,...,...,...,...,...,...
14377,97.548.167/0001-23,9718000.83,2022-10-15,60.444.437/0001-46,Debêntures - Debênture simples,13041906526.46,0.075
14378,97.548.167/0001-23,9668253.90,2022-06-28,33.014.556/0001-96,Títulos de Crédito Privado - Nota Promissória/...,13041906526.46,0.074
14379,32.313.932/0001-80,11842659.06,2029-09-26,15.009.178/0001-70,Debêntures - Debênture simples,51412412.93,23.035
14380,32.313.932/0001-80,14545081.95,2022-12-31,15.009.178/0001-70,Debêntures - Debênture simples,51412412.93,28.291


In [78]:
# Verifica se não há valores ausentes em CPF_CNPJ_EMISSOR 
df = aplic_cda6[aplic_cda6['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

True


#### Investimento no Exterior

In [79]:
aplic_cda7 = pd.merge(cda7, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda7['PR_PATRIM_LIQ'] = aplic_cda7.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda7 = aplic_cda7.drop(['TP_APLIC'], axis=1)

aplic_cda7 = aplic_cda7.fillna('')
aplic_cda7

,CNPJ_FUNDO,VL_APLIC,CD_ATIVO,BV_MERC,PAIS,DT_VENC,EMISSOR,DS_ATIVO,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,00.083.181/0001-67,39697260.92,OPPTEMP1,Outras bolsas ou mercados de balcão organizados,ESTADOS UNIDOS DA AMÉRICA,,TEMPUS OPP 1 CLASS,Investimento no Exterior - Outros,15519640726.95,0.256
1,00.102.322/0001-41,8196096.35,BABA US,NYSE EURONEXT,ESTADOS UNIDOS DA AMÉRICA,,ALIBABA GROUP HOLDING LTD,Investimento no Exterior - Ação ordinária,71548033.21,11.455
2,00.102.322/0001-41,5854514.16,AMZN,NASDAQ STOCK MARKET,ESTADOS UNIDOS DA AMÉRICA,,AMAZON.COM INC,Investimento no Exterior - Ação ordinária,71548033.21,8.183
3,00.102.322/0001-41,11049923.85,JPMC,NYSE EURONEXT,ESTADOS UNIDOS DA AMÉRICA,,JP MORGAN CHASE CO.,Investimento no Exterior - Ação ordinária,71548033.21,15.444
4,00.360.293/0001-18,205976213.92,EJ9011747,Outras bolsas ou mercados de balcão organizados,BRASIL,2025-01-07,GOVERNO BRASILEIRO,Investimento no Exterior - Título da Dívida Ex...,285328701.61,72.189
...,...,...,...,...,...,...,...,...,...,...
16414,97.525.430/0001-69,99290.55,SHOP US,NYSE EURONEXT,ESTADOS UNIDOS DA AMÉRICA,,SHOPIFY INC,Investimento no Exterior - Depository Receipt ...,21398864.37,0.464
16415,97.525.430/0001-69,128820.83,SQ US,NYSE EURONEXT,ESTADOS UNIDOS DA AMÉRICA,,SQUARE INC,Investimento no Exterior - Depository Receipt ...,21398864.37,0.602
16416,97.525.430/0001-69,78788.65,TDOC US,NYSE EURONEXT,ESTADOS UNIDOS DA AMÉRICA,,TELADOC HEALTH INC,Investimento no Exterior - Depository Receipt ...,21398864.37,0.368
16417,97.525.430/0001-69,103488.24,700 HK,HONG KONG EXCHANGES AND CLEARING,ESTADOS UNIDOS DA AMÉRICA,,TENCENT HOLDINGS LTD,Investimento no Exterior - Depository Receipt ...,21398864.37,0.484


In [80]:
# Verifica se não há valores ausentes em EMISSOR (Nome do Emissor) 
df = aplic_cda7[aplic_cda7['EMISSOR']=='']
print(df.empty) 

False


#### Atos Não Codificados

In [81]:
aplic_cda8 = pd.merge(cda8, cda0,
                        how="inner", on=["CNPJ_FUNDO"])

aplic_cda8['PR_PATRIM_LIQ'] = aplic_cda8.apply(lambda row: pr_patrim_liq(row), axis=1)
aplic_cda8 = aplic_cda8.drop(['TP_APLIC'], axis=1)

aplic_cda8 = aplic_cda8.fillna('')
aplic_cda8

,CNPJ_FUNDO,VL_APLIC,DS_ATIVO,CPF_CNPJ_EMISSOR,EMISSOR,VL_PATRIM_LIQ,PR_PATRIM_LIQ
0,06.537.068/0001-90,0.00,Cotas de fundos de renda fixa - Cotas de fundo...,,,1325.12,0.000
1,06.537.068/0001-90,1323.62,Disponibilidades - Outros - VALOR DAS DISPONIB...,,,1325.12,99.887
2,06.537.068/0001-90,0.00,Valores a pagar - Outros - VALOR A PAGAR,,,1325.12,0.000
3,07.408.147/0001-64,0.00,Cotas de fundos de renda fixa - Cotas de fundo...,,,28308.60,0.000
4,07.408.147/0001-64,28293.61,Disponibilidades - Outros - VALOR DAS DISPONIB...,,,28308.60,99.947
...,...,...,...,...,...,...,...
161099,11.179.262/0001-81,4.75,Valores a pagar - Outros - SELIC A PAGAR,,,1780896.83,-0.000
161100,11.179.262/0001-81,424.66,Valores a pagar - Outros - TAXA ADMIN PROVISAO,,,1780896.83,-0.024
161101,11.179.262/0001-81,113.25,Valores a pagar - Outros - TAXA CUSTOD PROVISAO,,,1780896.83,-0.006
161102,11.179.262/0001-81,3822.03,Valores a pagar - Outros - TAXA GESTAO PROVISAO,,,1780896.83,-0.215


In [82]:
# Verifica se não há valores ausentes em CPF_CNPJ_EMISSOR 
df = aplic_cda8[aplic_cda8['CPF_CNPJ_EMISSOR']=='']
print(df.empty) 

False


In [83]:
# Verifica se não há valores ausentes em EMISSOR (Nome do Emissor) 
df = aplic_cda8[aplic_cda8['EMISSOR']=='']
print(df.empty) 

False


In [84]:
aplic_cda8[aplic_cda8['CPF_CNPJ_EMISSOR']!='']

,CNPJ_FUNDO,VL_APLIC,DS_ATIVO,CPF_CNPJ_EMISSOR,EMISSOR,VL_PATRIM_LIQ,PR_PATRIM_LIQ
6,02.010.153/0001-45,54712.12,Disponibilidades - Outros - BANCO BRASIL CAIXA,00.000.000/0001-91,BCO DO BRASIL,17079539.69,0.320
7,02.010.153/0001-45,-1150.70,Mercado Futuro - Posições compradas - Contrato...,54.641.030/0001-06,BMF - BOLSA DE MERCADORIAS E FUTUROS,17079539.69,-0.007
8,02.010.153/0001-45,0.00,Mercado Futuro - Posições compradas - Contrato...,54.641.030/0001-06,BMF - BOLSA DE MERCADORIAS E FUTUROS,17079539.69,0.000
9,02.010.153/0001-45,-230.88,Mercado Futuro - Posições compradas - Contrato...,54.641.030/0001-06,BMF - BOLSA DE MERCADORIAS E FUTUROS,17079539.69,-0.001
10,02.010.153/0001-45,2186.10,Mercado Futuro - Posições vendidas - Contrato ...,54.641.030/0001-06,BMF - BOLSA DE MERCADORIAS E FUTUROS,17079539.69,0.013
...,...,...,...,...,...,...,...
161090,11.179.262/0001-81,1775.33,Valores a pagar - Outros - CINEHOLLIUDY,19.361.060/0001-68,FUNCINE FUNDO FINANC INDUSTRIA CINEMATOG,1780896.83,-0.100
161092,11.179.262/0001-81,5404.19,Valores a pagar - Outros - DAS,19.361.060/0001-68,FUNCINE FUNDO FINANC INDUSTRIA CINEMATOG,1780896.83,-0.303
161093,11.179.262/0001-81,1827.14,Valores a pagar - Outros - DAS AMORT,19.361.060/0001-68,FUNCINE FUNDO FINANC INDUSTRIA CINEMATOG,1780896.83,-0.103
161096,11.179.262/0001-81,66258.28,Valores a pagar - Outros - ELA DISSE,19.361.060/0001-68,FUNCINE FUNDO FINANC INDUSTRIA CINEMATOG,1780896.83,-3.721


## Exportação dos dados

### Mapeamento do Google Drive

In [ ]:
drive.mount('/content/drive/')

In [86]:
!mkdir -p /content/drive/MyDrive/carteira_fundos/entrada
!mkdir -p /content/drive/MyDrive/carteira_fundos/saida

In [87]:
# Salva no Google Drive os arquivos baixados do Portal de Dados Abertos CVM e site B3
!cp cad_fi.csv /content/drive/MyDrive/carteira_fundos/entrada
!cp cda_fi.zip /content/drive/MyDrive/carteira_fundos/entrada
!cp isinp.zip /content/drive/MyDrive/carteira_fundos/entrada

In [88]:
# Define diretório do Google Drive para salvar o resultado do processamento
diretorio_saida = '/content/drive/MyDrive/carteira_fundos/saida/'

### Fundos

In [89]:
fundos.to_csv(diretorio_saida + 'fundos.csv', index=False, encoding='utf-8')
fundos.head()

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,SIT,DT_INI_SIT,FUNDO_COTAS,FUNDO_EXCLUSIVO,VL_PATRIM_LIQ
0,FAPI,02.177.812/0001-32,ITAU FAPI RENDA FIXA - FUNDO DE APOSENTADORIA ...,EM FUNCIONAMENTO NORMAL,1997-11-27,,,66438968.44
1,FAPI,02.226.122/0001-26,UNIBANCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,EM FUNCIONAMENTO NORMAL,1997-12-05,,,4740.45
2,FAPI,02.177.815/0001-76,ITAU FAPI CONSERVADOR - FUNDO DE APOSENTADORIA...,EM FUNCIONAMENTO NORMAL,1997-12-16,,,2044493.92
3,FAPI,02.185.027/0001-21,BRADESCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,EM FUNCIONAMENTO NORMAL,1997-12-19,,,453086833.64
4,FAPI,02.297.533/0001-02,BRADESCO FUNDO DE APOSENTADORIA PROGRAMADA IND...,EM FUNCIONAMENTO NORMAL,1997-12-29,,,15654242.81


### Gestores

In [90]:
fundos_gestores.to_csv(diretorio_saida + 'fundos_gestores.csv', index=False, encoding='utf-8')
fundos_gestores.head()

,CNPJ_FUNDO,CPF_CNPJ_GESTOR
0,03.915.495/0001-77,60.701.190/0001-04
1,02.838.578/0001-47,30.822.936/0001-69
2,02.998.275/0001-91,62.375.134/0001-44
3,03.555.959/0001-81,00.360.305/0001-04
4,03.859.784/0001-04,33.311.713/0001-25


In [91]:
gestores.to_csv(diretorio_saida + 'gestores.csv', index=False, encoding='utf-8')
gestores.head()

,CPF_CNPJ_GESTOR,GESTOR
60,60.770.336/0001-65,BANCO ALFA DE INVESTIMENTO S.A.
1070,60.701.190/0001-04,ITAU UNIBANCO S.A.
1095,33.172.537/0001-98,BANCO J.P. MORGAN S.A.
1707,12.695.840/0001-03,JULIUS BAER FAMILY OFFICE BRASIL GESTÃO DE PAT...
3277,62.375.134/0001-44,BRAM - BRADESCO ASSET MANAGEMENT S.A. DISTRIBU...


### Ativos

In [92]:
ativos_emissor_isin = pd.concat([titulos_publicos_emissor, 
           swap_emissor,       
           ativos_codificados_emissor]).fillna('')

ativos_emissor_isin.reset_index(drop=True, inplace=True)       

ativos_emissor_isin['ID_EMISSOR'] = ativos_emissor_isin.apply(lambda row: id_emissor(row), axis=1)
ativos_emissor_isin['ID_EMISSOR'] = ativos_emissor_isin['ID_EMISSOR'].apply(hash)

ativos_emissor_isin['ID_ATIVO']   =  \
        ativos_emissor_isin['CD_ATIVO'] \
+ '|' + ativos_emissor_isin['CD_ISIN'] \
+ '|' \
+ '|' \
+ '|' \
+ '|' \
+ '|' 

ativos_emissor_isin['ID_ATIVO'] = ativos_emissor_isin['ID_ATIVO'].apply(hash)

In [93]:
ativos_emissor = pd.concat([depositos_prazo_emissor, 
           titulos_agronegocio_emissor, 
           investimento_exterior_emissor, 
           ativos_nao_codificados_emissor]).fillna('')

ativos_emissor.reset_index(drop=True, inplace=True)       

ativos_emissor['ID_EMISSOR'] = ativos_emissor.apply(lambda row: id_emissor(row), axis=1)
ativos_emissor['ID_EMISSOR'] = ativos_emissor['ID_EMISSOR'].apply(hash)

ativos_emissor['ID_ATIVO']   = \
        ativos_emissor['CD_ATIVO'] \
+ '|' \
+ '|' + ativos_emissor['DS_ATIVO'] \
+ '|' + ativos_emissor['DT_VENC'] \
+ '|' + ativos_emissor['BV_MERC'] \
+ '|' + ativos_emissor['PAIS'] \
+ '|' + ativos_emissor['ID_EMISSOR'] 
           
ativos_emissor['ID_ATIVO'] = ativos_emissor['ID_ATIVO'].apply(hash)

In [94]:
ativos = pd.concat([ativos_emissor_isin[['ID_ATIVO','CD_ATIVO','CD_ISIN','DS_ATIVO', 'DT_VENC','ID_EMISSOR']],
                                     ativos_emissor[['ID_ATIVO','CD_ATIVO','DS_ATIVO','BV_MERC','PAIS','DT_VENC','ID_EMISSOR']]]).fillna('')

df = ativos.drop(['ID_EMISSOR'], axis=1)
df.to_csv(diretorio_saida + 'ativos.csv', index=False, encoding='utf-8')
df.head()

,ID_ATIVO,CD_ATIVO,CD_ISIN,DS_ATIVO,DT_VENC,BV_MERC,PAIS
0,1d10237b4abbca11a03a6ee745d6cac6,,BRSTNCLF1R74,LETRAS FINANCEIRAS DO TESOURO LETRAS FINANCEIR...,2022-09-01,,
1,c72352b69567bd78998644887a498ae9,,BRSTNCNTF1P8,NOTAS DO TESOURO NACIONAL SERIE F NOTAS DO TES...,2027-01-01,,
2,266c59d25d42be89f4db5e419fbc4ef4,,BRSTNCLTN7O0,LETRAS DO TESOURO NACIONAL LETRAS DO TESOURO N...,2022-07-01,,
3,744a9a382e0cdeae5d5711538e0b8dbc,,BRSTNCNTB4O9,NOTAS DO TESOURO NACIONAL SERIE B NOTAS DO TES...,2023-05-15,,
4,931fda6a5040d49705a8baf564d28659,,BRSTNCNTB096,NOTAS DO TESOURO NACIONAL SERIE B NOTAS DO TES...,2024-08-15,,


In [95]:
df = ativos[['ID_ATIVO','ID_EMISSOR']]

ativos_emissores = df[df['ID_EMISSOR']!=''].drop_duplicates()

ativos_emissores.to_csv(diretorio_saida + 'ativos_emissores.csv', index=False, encoding='utf-8')
ativos_emissores.head()

,ID_ATIVO,ID_EMISSOR
0,1d10237b4abbca11a03a6ee745d6cac6,0425aba961b0e9399dbc2cfb7628f351
1,c72352b69567bd78998644887a498ae9,0425aba961b0e9399dbc2cfb7628f351
2,266c59d25d42be89f4db5e419fbc4ef4,0425aba961b0e9399dbc2cfb7628f351
3,744a9a382e0cdeae5d5711538e0b8dbc,0425aba961b0e9399dbc2cfb7628f351
4,931fda6a5040d49705a8baf564d28659,0425aba961b0e9399dbc2cfb7628f351


### Emissores

In [96]:
df = nomes_emissores.fillna('')

df['ID_EMISSOR'] = df.apply(lambda row: id_emissor(row), axis=1)
df['ID_EMISSOR'] = df['ID_EMISSOR'].apply(hash)

emissores = pd.merge(df, ativos_emissores.drop(['ID_ATIVO'], axis=1),
                                    how="inner", on=["ID_EMISSOR"])

emissores = emissores.drop_duplicates()

In [97]:
emissores.to_csv(diretorio_saida + 'emissores.csv', index=False, encoding='utf-8')
emissores.head()

,CPF_CNPJ_EMISSOR,EMISSOR,ID_EMISSOR
0,07.810.517/0001-95,BANCO BTG PACTUAL S/A,249b405a038d00c48f7483c43b59e5a0
3,18.236.120/0001-58,NU PAGAMENTOS,a56f712fdc592e2c9d4298a2749e82fc
5,30.121.792/0001-13,CREDIT SUISSE,19cd2d95762278159bf15475ccfe4302
68,60.701.190/4816-09,ITAU UNIBANCO S,df1ffeef2ddbe3968250b491c38abb3c
72,11.476.673/0001-39,BANCO RANDON SA,76321e15ed003bb6e20e7a2beb2324d1


### Aplicações

#### Aplicações em Ativos

In [98]:
aplicacoes_ativos_isin = pd.concat([aplic_cda1, aplic_cda3, aplic_cda4]).fillna('')

aplicacoes_ativos_isin.reset_index(drop=True, inplace=True)  

aplicacoes_ativos_isin['ID_ATIVO'] = \
        aplicacoes_ativos_isin['CD_ATIVO'] \
+ '|' + aplicacoes_ativos_isin['CD_ISIN'] \
+ '|' \
+ '|' \
+ '|' \
+ '|' \
+ '|' 

aplicacoes_ativos_isin['ID_ATIVO'] = aplicacoes_ativos_isin['ID_ATIVO'].apply(hash)

In [99]:
aplicacoes_ativos = pd.concat([aplic_cda5, aplic_cda6, aplic_cda7, aplic_cda8]).fillna('')

aplicacoes_ativos.reset_index(drop=True, inplace=True) 

aplicacoes_ativos['ID_ATIVO'] = \
        aplicacoes_ativos['CD_ATIVO'] \
+ '|' \
+ '|' + aplicacoes_ativos['DS_ATIVO'] \
+ '|' + aplicacoes_ativos['DT_VENC'] \
+ '|' + aplicacoes_ativos['BV_MERC'] \
+ '|' + aplicacoes_ativos['PAIS'] \
+ '|' + aplicacoes_ativos.apply(lambda row: id_emissor(row), axis=1).apply(hash) 

aplicacoes_ativos['ID_ATIVO'] = aplicacoes_ativos['ID_ATIVO'].apply(hash)

In [100]:
aplicacoes_ativos_completo = pd.concat([aplicacoes_ativos_isin[['CNPJ_FUNDO','VL_APLIC','PR_PATRIM_LIQ','ID_ATIVO']],
                                 aplicacoes_ativos[['CNPJ_FUNDO','VL_APLIC','PR_PATRIM_LIQ','ID_ATIVO']]]).\
                                     fillna('')

aplicacoes_ativos_completo.reset_index(drop=True, inplace=True)
aplicacoes_ativos_completo['ID_APLIC'] = aplicacoes_ativos_completo.index

In [101]:
aplicacoes_ativos_completo.to_csv(diretorio_saida + 'aplicacoes_ativos.csv', index=False, encoding='utf-8')
aplicacoes_ativos_completo.head()

,CNPJ_FUNDO,VL_APLIC,PR_PATRIM_LIQ,ID_ATIVO,ID_APLIC
0,02.010.153/0001-45,856146.22,5.013,1d10237b4abbca11a03a6ee745d6cac6,0
1,02.010.153/0001-45,4355691.07,25.502,c72352b69567bd78998644887a498ae9,1
2,02.010.153/0001-45,7055241.89,41.308,266c59d25d42be89f4db5e419fbc4ef4,2
3,02.010.153/0001-45,26512.28,0.155,744a9a382e0cdeae5d5711538e0b8dbc,3
4,02.010.153/0001-45,1265516.96,7.410,931fda6a5040d49705a8baf564d28659,4


#### Aplicações em Fundos

In [102]:
aplicacoes_cotas_fundos = aplic_cda2[['CNPJ_FUNDO','VL_APLIC','PR_PATRIM_LIQ','CNPJ_FUNDO_COTA']]

In [103]:
aplicacoes_cotas_fundos.to_csv(diretorio_saida + 'aplicacoes_cotas_fundos.csv', index=False, encoding='utf-8')
aplicacoes_cotas_fundos.head()

,CNPJ_FUNDO,VL_APLIC,PR_PATRIM_LIQ,CNPJ_FUNDO_COTA
0,00.068.305/0001-35,46221487.84,100.077,08.046.417/0001-05
1,00.071.477/0001-68,33160782612.92,100.008,01.608.573/0001-65
2,00.083.181/0001-67,147100863.98,0.948,01.235.622/0001-61
3,00.083.181/0001-67,216970194.57,1.398,10.597.218/0001-29
4,00.083.181/0001-67,1915404901.42,12.342,08.277.553/0001-06
